In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import tensorflow as tf


In [5]:
# Ensure output folder exists
os.makedirs("/content/drive/MyDrive/grace_tensorflow_git/models", exist_ok=True)

In [6]:
# Load dataset
data_path = "/content/drive/MyDrive/grace_tensorflow_git/data/processed/training_dataset.csv"
df = pd.read_csv(data_path)
df = df.drop(columns=['time', 'number'])


In [7]:
# Load feature importance
importance_path = "/content/drive/MyDrive/grace_tensorflow_git/data/processed/Feature_Importance.csv"
importance_df = pd.read_csv(importance_path)
selected_features = importance_df[importance_df['Ranking'] == 1]['Feature'].tolist()

In [8]:
# Target and extra features
target = "lwe_thickness"
extra_features = ["year", "month", "lon", "lat"]

features = selected_features + extra_features
features = [f for f in features if f != target]

In [9]:
# Prepare X and y
X = df[features].values

y = df[target].values

In [10]:
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [11]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Reshape for CNN (samples, height, width, channels) - simple 1D reshaped as 2D
X_train_cnn = X_train.reshape(-1, X_train.shape[1], 1, 1)
X_test_cnn = X_test.reshape(-1, X_test.shape[1], 1, 1)

In [13]:
# Build simple CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_cnn.shape[1], 1, 1)),
    tf.keras.layers.Conv2D(16, (3, 1), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(32, (3, 1), activation='relu', padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # regression output
])

In [14]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 9, 1, 16)       │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 9, 1, 32)       │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,193 (78.88 KB)

 Trainable params: 20,193 (78.88 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Callbacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint("/content/drive/MyDrive/grace_tensorflow_git/models/simple_cnn_model.h5", save_best_only=True)

In [ ]:
# Train model
history = model.fit(
    X_train_cnn, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

Epoch 1/100
6152/6164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 20.1593 - mae: 3.3100

6164/6164 ━━━━━━━━━━━━━━━━━━━━ 27s 4ms/step - loss: 20.1512 - mae: 3.3093 - val_loss: 13.6187 - val_mae: 2.7251
Epoch 2/100
6156/6164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 12.8428 - mae: 2.6621

6164/6164 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - loss: 12.8422 - mae: 2.6620 - val_loss: 11.4581 - val_mae: 2.4942
Epoch 3/100
6154/6164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 11.2392 - mae: 2.4901

6164/6164 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 11.2389 - mae: 2.4900 - val_loss: 10.6980 - val_mae: 2.4177
Epoch 4/100
6156/6164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.3491 - mae: 2.3851

6164/6164 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 10.3488 - mae: 2.3851 - val_loss: 9.7205 - val_mae: 2.3026
Epoch 5/100
6154/6164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.4812 - mae: 2.2728

...

6164/6164 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - loss: 2.7826 - mae: 1.2170 - val_loss: 2.9832 - val_mae: 1.2547
Epoch 85/100
6164/6164 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - loss: 2.7911 - mae: 1.2161 - val_loss: 3.0463 - val_mae: 1.2617
Epoch 86/100
6155/6164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.7781 - mae: 1.2133

6164/6164 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 2.7782 - mae: 1.2133 - val_loss: 2.9651 - val_mae: 1.2374
Epoch 87/100
6164/6164 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - loss: 2.7559 - mae: 1.2108 - val_loss: 2.9741 - val_mae: 1.2540
Epoch 88/100
6164/6164 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 2.7057 - mae: 1.1963 - val_loss: 2.9822 - val_mae: 1.2487
Epoch 89/100
6164/6164 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 2.7370 - mae: 1.2058 - val_loss: 3.0785 - val_mae: 1.2716
Epoch 90/100
6164/6164 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - loss: 2.7417 - mae: 1.2040 - val_loss: 3.0726 - val_mae: 1.2583
Epoch 91/100
6153/6164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.7033 - mae: 1.1974

6164/6164 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 2.7033 - mae: 1.1974 - val_loss: 2.9541 - val_mae: 1.2477
Epoch 92/100
6145/6164 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.6886 - mae: 1.1946

6164/6164 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 2.6886 - mae: 1.1946 - val_loss: 2.8330 - val_mae: 1.2117
Epoch 93/100
6164/6164 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - loss: 2.6901 - mae: 1.1954 - val_loss: 2.9963 - val_mae: 1.2708
Epoch 94/100
6164/6164 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 2.6714 - mae: 1.1928 - val_loss: 2.9918 - val_mae: 1.2717
Epoch 95/100
6164/6164 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - loss: 2.6367 - mae: 1.1828 - val_loss: 3.2860 - val_mae: 1.2970
Epoch 96/100
6164/6164 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - loss: 2.6345 - mae: 1.1824 - val_loss: 2.9147 - val_mae: 1.2349
Epoch 97/100
6164/6164 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - loss: 2.6392 - mae: 1.1837 - val_loss: 2.8676 - val_mae: 1.2216
Epoch 98/100
6164/6164 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - loss: 2.6086 - mae: 1.1759 - val_loss: 2.8866 - val_mae: 1.2270
Epoch 99/100
6164/6164 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - loss: 2.5733 - mae: 1.1711 - val_loss: 2.8416 - val_mae: 1.2166
Epoch 100/100
6160/6164 ━━━━━

6164/6164 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - loss: 2.5945 - mae: 1.1735 - val_loss: 2.8323 - val_mae: 1.2274


In [3]:
# Plot training curves
plt.figure(figsize=(8,6))
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.title('CNN Training Curves')
plt.legend()
plt.grid(True)
plt.savefig("/content/drive/MyDrive/grace_tensorflow_git/models/simple_cnn_training_curve_TEST.png", dpi=300)
plt.show()

NameError: name 'history' is not defined

<Figure size 800x600 with 0 Axes>

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
# Predictions and metrics
y_pred = model.predict(X_test_cnn).flatten()
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
bias = np.mean(y_pred - y_test)  # mean error
corr = np.corrcoef(y_test, y_pred)[0, 1]  # Pearson correlation coefficient
print(f"Test MAE: {mae}")
print(f"Test RMSE: {rmse}")
metrics_df = pd.DataFrame({
    'MAE': [mae],
    'RMSE': [rmse],
    'R2': [r2],
    'Bias': [bias],
    'Corr': [corr]
})

print("CNN Evaluation Metrics:")
print(metrics_df)

1927/1927 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Test MAE: 1.2039213688572816
Test RMSE: 1.6704104780544762
CNN Evaluation Metrics:
        MAE     RMSE        R2     Bias      Corr
0  1.203921  1.67041  0.946498 -0.09338  0.972984


In [ ]:
 # --- User input for year and month ---
year_input = int(input("Enter the year for prediction: "))
month_input = int(input("Enter the month for prediction: "))

era5_path = "/content/drive/MyDrive/grace_tensorflow_git/data/processed/ERA5_Greece.csv"
era5_df = pd.read_csv(era5_path)

# Keep the same features as training
subset = era5_df[(era5_df['year'] == year_input) & (era5_df['month'] == month_input)]
X_subset = subset[features].values


Enter the year for prediction: 2025
Enter the month for prediction: 5


In [ ]:
X_subset = scaler.transform(X_subset)
X_subset_cnn = X_subset.reshape(-1, X_subset.shape[1], 1, 1)

In [ ]:
# Predictions
predictions = model.predict(X_subset_cnn).flatten()

351/351 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [ ]:
# Export predictions as GeoTIFF
# Assume lon/lat are in subset
lon = subset['lon'].values
lat = subset['lat'].values

In [ ]:
# Create 2D grid for raster
unique_lon = np.sort(np.unique(lon))
unique_lat = np.sort(np.unique(lat))[::-1]  # flip for north-to-south

grid = predictions.reshape(len(unique_lat), len(unique_lon))

In [ ]:
!pip -q install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 32.8 MB/s eta 0:00:00


In [ ]:
import rasterio
from rasterio.transform import from_origin

In [ ]:
# Geo transform
pixel_size_lon = (unique_lon[-1] - unique_lon[0]) / (len(unique_lon) - 1)
pixel_size_lat = (unique_lat[0] - unique_lat[-1]) / (len(unique_lat) - 1)
transform = from_origin(unique_lon[0] - pixel_size_lon/2,
                        unique_lat[0] + pixel_size_lat/2,
                        pixel_size_lon,
                        pixel_size_lat)

output_tif = f"/content/drive/MyDrive/grace_tensorflow_git/data/predictions/predictions_{year_input}_{month_input}.tif"
with rasterio.open(
    output_tif,
    'w',
    driver='GTiff',
    height=grid.shape[0],
    width=grid.shape[1],
    count=1,
    dtype=grid.dtype,
    crs='EPSG:4326',
    transform=transform
) as dst:
    dst.write(grid, 1)

print(f"✅ Predictions exported to {output_tif}")

✅ Predictions exported to /content/drive/MyDrive/grace_tensorflow_git/data/predictions/predictions_2025_5.tif
